<a href="https://colab.research.google.com/github/Lyynn777/Big-Data-Customer-Churn-Prediction-Using-PySpark-and-Machine-Learning/blob/main/Churn_Prediction_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark==3.5.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425346 sha256=8f953a7f85b604e2b628b6a991dc7bcc76a3c6059dfd2473986560312a81093e
  Stored in directory: /root/.cache/pip/wheels/84/40/20/65eefe766118e0a8f8e385cc3ed6e9eb7241c7e51cfc04c51a
Successfully built pyspark
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.1
    Uninstalling pyspark-3.5.1:
      Successfully uninstalled pyspark-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dataproc-spark-connect 0.8.3 requires pyspark[connect]~=3.5.1, but you have pyspark 3.5.0 which is incompatible.


In [2]:
import pyspark
from pyspark.sql import SparkSession

print("PySpark version:", pyspark.__version__)


PySpark version: 3.5.0


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BigDataAnalytics_Churn") \
    .getOrCreate()

spark


In [4]:
data = [("Alice", 34), ("Bob", 45), ("Charlie", 29)]
columns = ["name", "age"]

df_test = spark.createDataFrame(data, columns)
df_test.show()
df_test.printSchema()


+-------+---+
|   name|age|
+-------+---+
|  Alice| 34|
|    Bob| 45|
|Charlie| 29|
+-------+---+

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [9]:
!wget -q https://raw.githubusercontent.com/easonlai/telco_customer_churn/main/data/WA_Fn-UseC_-Telco-Customer-Churn_R2.csv -O churn2.csv


In [10]:
!head -n 10 churn2.csv

customerID,gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7590-VHVEG,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,1,2,29.85,29.85,0
5575-GNVDE,1,0,0,0,34,1,0,1,1,0,1,0,0,0,1,0,3,56.95,1889.5,0
3668-QPYBK,1,0,0,0,2,1,0,1,1,1,0,0,0,0,0,1,3,53.85,108.15,1
7795-CFOCW,1,0,0,0,45,0,0,1,1,0,1,1,0,0,1,0,0,42.3,1840.75,0
9237-HQITU,0,0,0,0,2,1,0,2,0,0,0,0,0,0,0,1,2,70.7,151.65,1
9305-CDSKC,0,0,0,0,8,1,1,2,0,0,1,0,1,1,0,1,2,99.65,820.5,1
1452-KIOVK,1,0,0,1,22,1,1,2,0,1,0,0,1,0,0,1,1,89.1,1949.4,0
6713-OKOMC,0,0,0,0,10,0,0,1,1,0,0,0,0,0,0,0,3,29.75,301.9,0
7892-POOKP,0,0,1,0,28,1,1,2,0,0,1,1,1,1,0,1,2,104.8,3046.05,1


In [12]:
df = spark.read.csv("churn2.csv", header=True, inferSchema=True)
df.show(5)
df.printSchema()


+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|Tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|7590-VHVEG|     0|            0|      1|         0|     1|           0|            0|              1|             0|           1|               0|          0|          0|              0|       0|               1| 

In [13]:
print("Rows:", df.count())
print("Columns:", len(df.columns))


Rows: 7011
Columns: 21


In [15]:
from pyspark.sql.functions import col, sum as _sum

missing = df.select([
    _sum(col(c).isNull().cast("int")).alias(c) for c in df.columns
])
missing.show()


+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|customerID|gender|SeniorCitizen|Partner|Dependents|Tenure|PhoneService|MultipleLines|InternetService|OnlineSecurity|OnlineBackup|DeviceProtection|TechSupport|StreamingTV|StreamingMovies|Contract|PaperlessBilling|PaymentMethod|MonthlyCharges|TotalCharges|Churn|
+----------+------+-------------+-------+----------+------+------------+-------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------+----------------+-------------+--------------+------------+-----+
|         0|     0|            0|      0|         0|     0|           0|            0|              0|             0|           0|               0|          0|          0|              0|       0|               0| 

In [16]:
from pyspark.sql.functions import when

df = df.withColumn(
    "TotalCharges",
    when(col("TotalCharges") == " ", None).otherwise(col("TotalCharges"))
)
df = df.withColumn("TotalCharges", col("TotalCharges").cast("float"))


In [17]:
df.select("TotalCharges").show(10)
df.printSchema()


+------------+
|TotalCharges|
+------------+
|       29.85|
|      1889.5|
|      108.15|
|     1840.75|
|      151.65|
|       820.5|
|      1949.4|
|       301.9|
|     3046.05|
|     3487.95|
+------------+
only showing top 10 rows

root
 |-- customerID: string (nullable = true)
 |-- gender: integer (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: integer (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- PhoneService: integer (nullable = true)
 |-- MultipleLines: integer (nullable = true)
 |-- InternetService: integer (nullable = true)
 |-- OnlineSecurity: integer (nullable = true)
 |-- OnlineBackup: integer (nullable = true)
 |-- DeviceProtection: integer (nullable = true)
 |-- TechSupport: integer (nullable = true)
 |-- StreamingTV: integer (nullable = true)
 |-- StreamingMovies: integer (nullable = true)
 |-- Contract: integer (nullable = true)
 |-- PaperlessBilling: integer (nullable = true)
 |--

In [21]:
for c in df.columns:
    print(f"Column: {c}")
    df.select(c).distinct().show(10)
    print("-"*50)


Column: customerID
+----------+
|customerID|
+----------+
|3668-QPYBK|
|6234-RAAPL|
|1894-IGFSG|
|6982-SSHFK|
|5859-HZYLF|
|6479-OAUSD|
|2592-YKDIF|
|6718-BDGHG|
|3195-TQDZX|
|4248-QPAVC|
+----------+
only showing top 10 rows

--------------------------------------------------
Column: gender
+------+
|gender|
+------+
|     1|
|     0|
+------+

--------------------------------------------------
Column: SeniorCitizen
+-------------+
|SeniorCitizen|
+-------------+
|            1|
|            0|
+-------------+

--------------------------------------------------
Column: Partner
+-------+
|Partner|
+-------+
|      1|
|      0|
+-------+

--------------------------------------------------
Column: Dependents
+----------+
|Dependents|
+----------+
|         1|
|         0|
+----------+

--------------------------------------------------
Column: Tenure
+------+
|Tenure|
+------+
|    31|
|    65|
|    53|
|    34|
|    28|
|    27|
|    26|
|    44|
|    12|
|    22|
+------+
only showing 

In [23]:
from pyspark.sql.functions import avg

overall_churn = df.agg(avg("Churn")).collect()[0][0]
print("Overall Churn Rate:", round(overall_churn, 4))


Overall Churn Rate: 0.2659


In [24]:
df.groupBy("gender") \
  .agg(avg("Churn").alias("churn_rate")) \
  .orderBy("gender") \
  .show()


+------+-------------------+
|gender|         churn_rate|
+------+-------------------+
|     0| 0.2697955657932623|
|     1|0.26201243640474847|
+------+-------------------+



In [25]:
df.groupBy("Contract") \
  .agg(avg("Churn").alias("churn_rate")) \
  .orderBy("Contract") \
  .show()


+--------+--------------------+
|Contract|          churn_rate|
+--------+--------------------+
|       0| 0.42749870533402384|
|       1| 0.11247443762781185|
|       2|0.028537455410225922|
+--------+--------------------+



In [26]:
from pyspark.sql.functions import when

df2 = df.withColumn(
    "TenureGroup",
    when(df.Tenure < 12, "0-12 months")
    .when(df.Tenure < 24, "12-24 months")
    .when(df.Tenure < 48, "24-48 months")
    .otherwise("48+ months")
)

df2.groupBy("TenureGroup") \
   .agg(avg("Churn").alias("churn_rate")) \
   .orderBy("TenureGroup") \
   .show()


+------------+-------------------+
| TenureGroup|         churn_rate|
+------------+-------------------+
| 0-12 months|0.48514369215781783|
|12-24 months|0.29586935638808837|
|24-48 months|0.20925925925925926|
|  48+ months|0.09621245102307358|
+------------+-------------------+



In [27]:
df.groupBy().avg("MonthlyCharges").show()

df.groupBy("Churn") \
  .agg(avg("MonthlyCharges").alias("avg_monthly_charge")) \
  .orderBy("Churn") \
  .show()


+-------------------+
|avg(MonthlyCharges)|
+-------------------+
|   64.7986449864499|
+-------------------+

+-----+------------------+
|Churn|avg_monthly_charge|
+-----+------------------+
|    0|61.302856032640165|
|    1|   74.451448497854|
+-----+------------------+



In [28]:
# Columns to use (exclude customerID)
feature_cols = [c for c in df.columns if c not in ["customerID", "Churn"]]

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

df_ml = assembler.transform(df).select("features", "Churn")
df_ml.show(5)


+--------------------+-----+
|            features|Churn|
+--------------------+-----+
|(19,[2,4,7,9,15,1...|    0|
|(19,[0,4,5,7,8,10...|    0|
|(19,[0,4,5,7,8,9,...|    1|
|(19,[0,4,7,8,10,1...|    0|
|(19,[4,5,7,15,16,...|    1|
+--------------------+-----+
only showing top 5 rows



In [29]:
train, test = df_ml.randomSplit([0.8, 0.2], seed=42)
train.count(), test.count()


(5673, 1338)

In [30]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    featuresCol="features",
    labelCol="Churn"
)

lr_model = lr.fit(train)


In [31]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Predictions
pred = lr_model.transform(test)

# Accuracy
acc_eval = MulticlassClassificationEvaluator(
    labelCol="Churn",
    predictionCol="prediction",
    metricName="accuracy"
)
accuracy = acc_eval.evaluate(pred)
print("Accuracy:", accuracy)

# ROC-AUC
auc_eval = BinaryClassificationEvaluator(
    labelCol="Churn",
    rawPredictionCol="rawPrediction",
    metricName="areaUnderROC"
)
auc = auc_eval.evaluate(pred)
print("ROC-AUC:", auc)


Accuracy: 0.8071748878923767
ROC-AUC: 0.8548697714835964


In [32]:
coefficients = lr_model.coefficients.toArray()
for col, coef in zip(feature_cols, coefficients):
    print(f"{col:20s} {coef}")


gender               -0.0505802453978815
SeniorCitizen        0.1958760418155304
Partner              0.026430869592271325
Dependents           -0.17182830940642294
Tenure               -0.06071983178283049
PhoneService         -0.40654019159944327
MultipleLines        0.3303181273841185
InternetService      1.317244997773876
OnlineSecurity       -0.3481546257985933
OnlineBackup         -0.12602740557166664
DeviceProtection     -0.0361111920403113
TechSupport          -0.2848722126548519
StreamingTV          0.39237748154815194
StreamingMovies      0.4366462518297136
Contract             -0.7087620697856416
PaperlessBilling     0.31577164650471395
PaymentMethod        0.0704027376298169
MonthlyCharges       -0.018065338705710744
TotalCharges         0.00034628524171564413
